# Big Data Project: Transforming Scientific Articles into Videos with Speech using Apache Spark and Kafka


In [ ]:
# make modules from py files auto-reload when changed
%load_ext autoreload
%autoreload 2

# Spark 

In [ ]:
#import findspark
import os
import sys
#from pyspark.sql import SparkSession

In [ ]:
sys.executable

In [ ]:
os.getcwd()

In [ ]:
# Set Spark environment variables
# os.environ['SPARK_HOME'] = '/path/to/spark'
# os.environ['PYSPARK_PYTHON'] = 'python'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
# os.environ['JAVA_HOME'] = '/path/to/java'

# Initialize findspark
#findspark.init()

# Create SparkSession and print version
# spark = SparkSession.builder.appName("PySpark Test").getOrCreate()
# print(f"Spark version: {spark.version}")
# spark.stop()

#TODO:   
create folders for data, checkpoints, output, dbg


# TTS 


## Basic run just for test 

In [ ]:
import torch
from datetime import datetime
from ArticleReader.Chunker import Chunker
from ArticleReader.LatexToSpeech import LatexParser
from ArticleReader.Narrator import Narrator
from Benchmarking import Bench
import pandas as pd 
import json

In [ ]:
input_file = "data/arXiv-2106.04624v1/main.tex"
output_file = "output/" + datetime.now().strftime(r"%y.%m.%d-%H")

parser = LatexParser()
content = parser.read_latex(input_file)
processed = parser.custom_latex_to_text(content)
parser.save_text(processed, "dbg/spec_my.txt")

tables = parser.get_tables()
parser.save_text(tables, "dbg/tables.tex")

In [ ]:
chunker = Chunker(max_len=200)
chunker.split_text_into_chunks(processed)
chunks = chunker.get_test_batch(10, 0)
# chunks = chunker.chunks
chunker.save_chunks_as_text(output_file + ".md", chunks)
print("text chunks:", [len(ch) for ch in chunks])

In [ ]:

narrator = Narrator()

# End